# Final clean up of the dataset

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import io
import statsmodels.api as sm
from scipy import stats
import geopandas as gpd

In [2]:
#Import the cleaned data for 2020 and 2021
file_path = 'C:/Users/Gaëlle/Documents/_CAS applied data science/3. Module 3 Data analysis and machine learning/Project/cleaned_dataset.xlsx'
df = pd.read_excel(file_path)

In [3]:
# Do they have the same variables?
print('the shape of the final dataset:', df.shape)

the shape of the final dataset: (301, 404)


In [4]:
column_names=df.columns.tolist()
#column_names

In [5]:
df.head(10)

,Nom_df2,G.1.4.P/Deliveries,G.1.5.P/Deliveries,G.1.6.P/Deliveries,G.1.7.P/Deliveries,I.1.16.M,A.5.1.F,A.7.14.M,L.6.4.F,KostAWLFA,...,F.2.13.V,A.1.11.M,KostStatA,K.1.3.F,D.1.11.M,H.4.4.M,I.1.13.M,E.4.16.M,StdBelP,F.3.1.V
0,Andreas Klinik,0.537190,0.446281,0.289256,0.157025,14,3,0,0,0.0,...,0,7,42564667.0,0,20,2,0,0,4093.87,0
1,Bethesda Spital AG,0.361747,0.288226,0.201385,0.086841,0,0,0,0,0.0,...,0,0,55361461.0,1,4,7,0,0,1657.00,0
2,CHUV Centre Hospitalier Universitaire Vaudois,0.279279,0.167754,0.104070,0.063684,12,192,19,69,30094276.0,...,4,313,696811983.0,8,275,28,0,2,0.00,21
3,Center da Sanda Engiadina Bassa Ospidal d'Engi...,0.253731,0.208955,0.164179,0.044776,0,1,0,0,428914.0,...,0,5,12237649.0,0,6,0,0,0,0.00,0
4,Centro Sanitario Valposchiavo Ospedale San Sisto,0.413793,0.275862,0.172414,0.103448,0,0,0,0,18082.0,...,0,5,4395777.0,0,16,0,0,0,0.00,0
5,Clinica Santa Chiara SA,0.356164,0.278539,0.164384,0.114155,3,1,0,0,0.0,...,0,21,24687076.0,2,37,1,0,0,0.00,0
6,Clinique Générale-Beaulieu,0.337248,0.270134,0.107383,0.162752,1,0,0,0,0.0,...,0,0,61241070.0,1,0,19,1,0,0.00,0
7,Clinique de La Source,0.429412,0.347059,0.194118,0.152941,7,82,0,0,0.0,...,0,22,66820513.0,0,8,11,0,0,0.00,0
8,Clinique des Grangettes SA,0.424550,0.340090,0.168919,0.171171,1,66,0,0,NaN,...,0,4,NaN,0,18,3,0,0,NaN,0
9,EHC Ensemble hospitalier de la Côte,0.291050,0.239605,0.155039,0.084567,6,28,0,0,0.0,...,0,81,88484895.0,0,100,13,0,0,0.00,1


## String variables

In [6]:
# Identify the variables whose format is character:
non_numerical_columns = df.select_dtypes(exclude=['number'])

In [7]:
print(non_numerical_columns.columns)

Index(['Nom_df2', 'Akt', 'SL', 'Typ', 'LA', 'WB', 'KT', 'RForm', 'Adr', 'Ort',
       'SA'],
      dtype='object')


String-related variables:
- Akt: Type d’activité: « Soins aigus » (A), « Psychiatrie » (P) et « Réadaptation / gériatrie » (R). Les maisons de naissance, indépendantes d’institutions cliniques (B), sont enregistrées sous leur propre type d’activité.
- SL: Prestations spécifiques: Indique si l’hôpital exploite un service d’urgence (NF) reconnu ou une unité de soins intensifs (IPS).
-Typ: Type de l’établissement: On distingue entre hôpitaux universitaires, hôpitaux de « prise en charge centralisée » (hôpitaux cantonaux), hôpitaux de « soins de base » et cliniques spécialisées (cliniques psychiatriques, cliniques de réadaptation, chirurgie, gynécologie/néonatologie, gériatrie, pédiatrie et cliniques spécialisées diverses) https://www.bfs.admin.ch/bfsstatic/dam/assets/227888/master
- LA: Offre de prestation de l’établissement.« Amb » : KS A.02.15 = 1; « Stat » : KS A.02.16 = 1
- WB: Formation de base et formation postgrade: Indique si l’hôpital dispose de places de formation de base ou de formation postgrade pour les étudiants en médecine (MSt), les médecins (Arzt) ou autres professionnels de la santé (BGs).
- KT: Canton responsable du point de vue administratif de l’hôpital.
- RForm: Forme juridique: « Raison individuelle / société » : KS A.03 = 1, 2 « SA / Sàrl » : KS A.03 = 3 à 7 « Association / fondation » : KS A.03 = 8 à 10 « Entreprises publiques » : KS A.03 = 17, 20 à 34
- Adr: adresse
- Ort: ville
- SA: Infrastructure spéciale: Indique si l’hôpital dispose d’appareils et d’équipements spéciaux. Parmi ceux-ci : la résonance magnétique nucléaire (MRI), la tomographie computérisée (CT), la tomographie par émission de positrons (PET), la caméra gamma, incluant la scintigraphie et le scanner SPECT (CC), l’accélérateur linéaire (LB) et le lithotripteur (LITO).

=> we could delete Adr and Ort

## Delete the unrelevant variables

In [8]:
# Delete uninteresting c-section rates, the address and the city
columns_to_drop = ['G.1.5.P/Deliveries', 'G.1.6.P/Deliveries', 'G.1.7.P/Deliveries', 'Adr', 'Ort']
df_1 = df.drop(columns=columns_to_drop)

## Normalize the variables per beds

### Variables that are percentages already therefore should not be normalized

In [9]:
columns_between_0_and_1 = []

In [10]:
# delete the string variables, but only for this short analysis
df_num = df.drop(columns=non_numerical_columns)


In [11]:
# What are the variables between 0 and 1 (assumption: they are percentages, therefore we can not divide them by the number of beds)
for column in df_num.columns:
    non_missing_values = df_num[column][df_num[column].notna()]
    if (non_missing_values >= 0).all() and (non_missing_values <= 1).all():
        columns_between_0_and_1.append(column)

In [12]:
print(columns_between_0_and_1)

['G.1.4.P/Deliveries', 'G.1.5.P/Deliveries', 'G.1.6.P/Deliveries', 'G.1.7.P/Deliveries', 'pPatLKP', 'L.5.6.F', 'L.1.2.F', 'pPatWAU', 'A.7.21.P', 'L.7.2.F', 'pPatHOK', 'pPatWAK']


Are these variables really percentages?
- the variables with the number of cases are not percentage but they might have only 1 case so the function above is identifing the variable as being a percentage which is not the case.

However, those are percentages:
- pPatLKP: Pourcentage de patients en division privée ou semi-privée
- pPatWAU: Pourcentage de patients avec domicile à l’étranger
- pPatHOK: Pourcentage de patients avec AMal obligatoire comme garante principale
- pPatWAK: Pourcentage de patients hors canton


Other percentages found in the handbook "Chiffres-clés des hôpitaux suisses"
- pBettenBelStatA : Taux d’occupation des lits
- ErlKVGStatVA: part des assureurs (%) Pourcentage du part des assureurs aux produits des prestations à la charge de l’AOS fournies aux patients hospitalisés, exclusivement assurés de base LAMal ; ventilés par type d’activité.
- ErlZvOKPStatVA: part des assureurs (%) Pourcentage du part des assureurs aux produits des prestations à la charge de l’AOS fournies aux patients hospitalisés, assurés en complémentaire ; ventilés par type d’activité.

=> we need to divide them by 100 because they are in %

=> We can not divide these variables per beds as they are already percentage!


In [13]:
# creation of 3 new variables to make them percentage from 0 to 1 as the rest of the percentages variables
df_1['pBettenBelStatA_new']=df_1['pBettenBelStatA']/100
df_1['ErlKVGStatVA_new']=df_1['ErlKVGStatVA']/100
df_1['ErlZvOKPStatVA_new']=df_1['ErlZvOKPStatVA']/100

In [14]:
# Delete the old variables
columns_to_drop = ['pBettenBelStatA', 'ErlKVGStatVA', 'ErlZvOKPStatVA']
df_2 = df_1.drop(columns=columns_to_drop)

### Normalize over the number of beds

In [15]:
# Are there hospitals with no informations on the number of beds? If so, we should delete the row:
count_na = (df_2['BettenStatA'].isna()).sum()
count_na

3

In [16]:
# 3 hospitals have no information on the number of beds, let's delete them
df_2 = df_2.dropna(subset=['BettenStatA'])

Variables that we cannot divide by the numbers of beds:
- the string variables
- JAHR
- G.1.4.P/Deliveries: our c-section rate
- pPatLKP
- pPatWAU
- pPatHOK
- pPatWAK
- pBettenBelStatA_new
- ErlKVGStatVA_new
- ErlZvOKPStatVA_new

In [17]:
# Exclude variables which will not be normalized
variables_to_exclude = ['Nom_df2', 'G.1.4.P/Deliveries', 'JAHR','Akt', 'SL', 'Typ', 'LA', 'WB', 'KT', 'RForm', 'SA', 'pPatLKP', 'pPatWAU', 'pPatHOK', 'pPatWAK', 'pBettenBelStatA_new', 'ErlKVGStatVA_new', 'ErlZvOKPStatVA_new']

# Variables which will be normalized
variables_to_divide = df_2.columns.difference(variables_to_exclude)

In [18]:
# Variable used to make the division: number of beds: BettenStatA
## La moyenne des lits à disposition pour l’exploitation correspond au nombre de journées-lits d’exploitation (jours d’exploitation des lits) divisé par le nombre de jour dans l’année (par 365 ou par 366 pour les années bissextiles); ventilée par type d’activité.

divisor_variable = 'BettenStatA'

In [19]:
# Division
df_2[variables_to_divide] = df_2[variables_to_divide].div(df_2[divisor_variable], axis=0)


In [20]:
df_2

,Nom_df2,G.1.4.P/Deliveries,I.1.16.M,A.5.1.F,A.7.14.M,L.6.4.F,KostAWLFA,L.5.5.F,A.7.1.F,D.2.6.P,...,K.1.3.F,D.1.11.M,H.4.4.M,I.1.13.M,E.4.16.M,StdBelP,F.3.1.V,pBettenBelStatA_new,ErlKVGStatVA_new,ErlZvOKPStatVA_new
0,Andreas Klinik,0.537190,0.250000,0.053571,0.000000,0.000000,0.000000,0.00000,0.000000,0.035714,...,0.000000,0.357143,0.035714,0.000000,0.000000,73.104821,0.000000,0.855577,0.450000,0.450000
1,Bethesda Spital AG,0.361747,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.011236,...,0.011236,0.044944,0.078652,0.000000,0.000000,18.617978,0.000000,0.758073,0.459425,0.469809
2,CHUV Centre Hospitalier Universitaire Vaudois,0.279279,0.013086,0.209378,0.020720,0.075245,32818.185387,0.00000,0.876772,0.093784,...,0.008724,0.299891,0.030534,0.000000,0.002181,0.000000,0.022901,0.828652,0.450000,0.450000
3,Center da Sanda Engiadina Bassa Ospidal d'Engi...,0.253731,0.000000,0.045455,0.000000,0.000000,19496.090909,0.00000,0.000000,0.045455,...,0.000000,0.272727,0.000000,0.000000,0.000000,0.000000,0.000000,0.729763,0.489951,0.729955
4,Centro Sanitario Valposchiavo Ospedale San Sisto,0.413793,0.000000,0.000000,0.000000,0.000000,1390.923077,0.00000,0.000000,0.000000,...,0.000000,1.230769,0.000000,0.000000,0.000000,0.000000,0.000000,0.838356,0.450000,0.674666
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
296,Stadtspital Triemli,0.372003,0.028060,0.964239,0.104587,0.028060,7868.002823,0.00000,2.066225,0.073976,...,0.002551,0.436203,0.104587,0.000000,0.007653,0.000000,0.025509,0.823716,0.450000,0.450000
297,Stiftung Spital Muri,0.335303,0.038835,0.000000,0.000000,0.000000,2128.495146,0.00000,0.000000,0.252427,...,0.019417,1.058252,0.087379,0.000000,0.000000,0.000000,0.000000,0.810799,0.450000,0.450000
298,Universitätsspital Basel,0.370635,0.032764,0.693732,0.029915,0.186610,29373.425926,0.00000,1.041311,0.034188,...,0.018519,0.356125,0.061254,0.009972,0.000000,0.000000,0.019943,0.849967,0.459653,0.453891
299,Universitätsspital Zürich,0.474082,0.000000,0.498316,0.022602,0.145297,24499.391048,0.00861,0.995556,0.022602,...,0.032288,0.228171,0.013992,0.000000,0.001076,0.000000,0.016144,0.736191,0.450000,0.450000


In [21]:
columns_to_drop = ['BettenStatA']
df_3 = df_2.drop(columns=columns_to_drop)

In [22]:
print('The shape of our final dataset is:', df_3.shape)

The shape of our final dataset is: (298, 398)


In [23]:
df_3.rename(columns={'G.1.4.P/Deliveries': 'c_section_rate', 'Nom_df2': 'name_hosp'}, inplace=True)


# New dataset with a new variable (c-section rate above or below 0.32)
## Create the threshold variable

In [29]:
df= pd.concat([df_3, df_3['c_section_rate'].apply(lambda x: 0 if x < 0.32 else 1).rename('threshold')], axis=1)

if the c-section rate is above 0.32% then the variable threshold gets 1, 0 otherwise

In [30]:
#from google.colab import drive
#drive.mount('/content/drive', force_remount=True)

In [31]:
#file_path = '/content/drive/MyDrive/Module 3/dataset_final_threshold.xlsx'
#df = pd.read_excel(file_path)

# Preprocessing data for Decision trees:

1. Dealing with NaN
2. Dealing with Strings

## 1. Dealing with NaN

In [32]:
nan_columns = df.columns[df.isnull().any()]
print("Columns with NaN values:")
print(nan_columns)
nan_sum = df[nan_columns].isnull().sum()
print("Sum of NaN values for each column:")
print(nan_sum)

Columns with NaN values:
Index(['KostAWLFA', 'KostForLFA', 'SL', 'EtDef', 'pCT_AMB', 'WB', 'pCT_STAT',
       'CMIb', 'PersAWB', 'CMIn', 'KostWBLFA', 'pMRI_AMB', 'pMRI_STAT', 'SA',
       'ErlKVGStatVA_new', 'ErlZvOKPStatVA_new'],
      dtype='object')
Sum of NaN values for each column:
KostAWLFA              1
KostForLFA             1
SL                    23
EtDef                 77
pCT_AMB               19
WB                     6
pCT_STAT              18
CMIb                  81
PersAWB               77
CMIn                  81
KostWBLFA              1
pMRI_AMB              46
pMRI_STAT             46
SA                    11
ErlKVGStatVA_new       4
ErlZvOKPStatVA_new    10
dtype: int64


**Variables that contain NaN values:**

KostAWLFA              1 - Kosten der universitären Lehre zur Ausbildung


---


KostForLFA             1 - Kosten der universitären Forschung

---


EtDef                 77 - Alle finanziellen Unterstützungen durch die öffentliche Hand oder privatrechtliche Organisationen wie Reserven, Gemeinde, Kantone, Bund, Private Rechtsträger in Form von allfälligen separat ausgewiesenen Defizitdeckungen.

---


pCT_AMB               19 - Anzahl CT Untersuchungen pro CT-Scanner, ambulant

---


pCT_STAT              18 - Anzahl CT Untersuchungen pro CT-Scanner, stationär


---


CMIb                  81 - Roher mittlerer Schweregrad der behandelten stationären Fälle, ohne Kompensation der Outlier, die eine Aufenthaltsdauer ausserhalb der erwarteten Grenzen haben; ausgewiesen pro Aktivitätstyp


---


PersAWB               77 - Ärzte in Weiterbildung



---


CMIn                  81 - Casemix Index (CMI) netto



---

KostWBLFA              1 - Kosten Weiterbildung



---


pMRI_AMB              46 - Anzahl MRI Untersuchungen pro MRI-Scanner, ambulant



---


pMRI_STAT             46 - Anzahl MRI Untersuchungen pro MRI-Scanner, stationär



---



ErlKVGStatVA_new       4 - Anteil der Versicherer am Erlös aus Leistungen zulasten der OKP an Patienten mit stationärem Aufenthalt



---



ErlZvOKPStatVA_new    10 - Anteil der Versicherer am Erlös aus Leistungen an zusatzversicherten Patienten mit stationärem Aufenthalt

**Procedure:** We now have much less missing values, however there are still some left. Let's check which hospitals have NaN values left.

In [33]:
for index, row in df.iterrows():
    nan_present = row.drop(['name_hosp', 'JAHR']).isnull().any()  # Check if any NaN value exists

    if nan_present:
        name_hosp_value = row['name_hosp']
        print(f"Row {index}: 'name_hosp' value is {name_hosp_value} - Contains NaN")

Row 0: 'name_hosp' value is Andreas Klinik - Contains NaN
Row 1: 'name_hosp' value is Bethesda Spital AG - Contains NaN
Row 2: 'name_hosp' value is CHUV Centre Hospitalier Universitaire Vaudois - Contains NaN
Row 3: 'name_hosp' value is Center da Sanda Engiadina Bassa Ospidal d'Engiadina Bassa - Contains NaN
Row 4: 'name_hosp' value is Centro Sanitario Valposchiavo Ospedale San Sisto - Contains NaN
Row 5: 'name_hosp' value is Clinica Santa Chiara SA - Contains NaN
Row 6: 'name_hosp' value is Clinique Générale-Beaulieu - Contains NaN
Row 7: 'name_hosp' value is Clinique de La Source - Contains NaN
Row 9: 'name_hosp' value is EHC Ensemble hospitalier de la Côte - Contains NaN
Row 10: 'name_hosp' value is EOC Ente ospedaliero cantonale - Contains NaN
Row 11: 'name_hosp' value is Etablissements Hospitaliers du Nord Vaudois (eHnv) - Contains NaN
Row 12: 'name_hosp' value is Flury Stiftung Spital Schiers - Contains NaN
Row 13: 'name_hosp' value is GZO Spital Wetzikon - Contains NaN
Row 14: '

**Procedure:** We now know that all the missing NaN values are from non-university hospitals. We will hence replace the NaN values by the mean of all the non-university hospitals.

In [34]:
means = df.drop('Typ', axis=1)[df['Typ'] != 'K111'].apply(pd.to_numeric, errors='coerce').mean()
print(means)

name_hosp                   NaN
c_section_rate         0.331909
I.1.16.M               0.088776
A.5.1.F                0.257695
A.7.14.M               0.009127
                         ...   
F.3.1.V                0.002374
pBettenBelStatA_new    0.763560
ErlKVGStatVA_new       0.466784
ErlZvOKPStatVA_new     0.574332
threshold              0.510714
Length: 398, dtype: float64


In [35]:
for col in means.index:
    df[col].fillna(means[col], inplace=True)

In [36]:
nan_columns = df.columns[df.isnull().any()]
print("Columns with NaN values:")
print(nan_columns)
nan_sum = df[nan_columns].isnull().sum()
print("Sum of NaN values for each column:")
print(nan_sum)

Columns with NaN values:
Index(['SL', 'WB', 'SA'], dtype='object')
Sum of NaN values for each column:
SL    23
WB     6
SA    11
dtype: int64


**Result:** We only have NaN in 3 string columns now, which we will deal with later.

## 2. String Value encoding

**Procedure:** Let's first check how many columns with string values we have

In [37]:
string_columns = df.select_dtypes(include='object').columns
print("Columns with string values:", string_columns)
for column in string_columns:
    print(f"Unique values and counts for column '{column}':")
    print(df[column].value_counts())
    print()

Columns with string values: Index(['name_hosp', 'Akt', 'SL', 'Typ', 'LA', 'WB', 'KT', 'RForm', 'SA'], dtype='object')
Unique values and counts for column 'name_hosp':
name_hosp
EOC Ente ospedaliero cantonale                                                 7
Spital Thurgau AG Kantonsspitäler Frauenfeld & Münsterlingen                   5
Spitalregion Rheintal Werdenberg Sarganserland                                 5
Luzerner Kantonsspital                                                         5
Spitäler Frutigen Meiringen Interlaken AG                                      5
                                                                              ..
Spitalverbund Appenzell Ausserrhoden, Akutsomatisches Spital Heiden/Herisau    1
Hirslanden Klinik St. Anna AG Hirslanden Klinik Meggen                         1
SMN - Klinik Villa im Park                                                     1
Spital Affoltern                                                               1
Spitalverbund

**Procedure:** We saw that the 'name_hosp' column takes many different values and most probably doesn't contain information with a lot of value for our prediction. We hence decide to drop this column and use 'one hot encoding' for the other string variables. One hot encoding creates binary columns for each category.

(This is better than using label encoder since label encoder assumes a clear, meaningful order or ranking in the variables to be encoded which is not the case for our string variables.)

In [38]:
df.shape

(298, 399)

In [39]:
df = df.drop('name_hosp', axis=1)
columns_to_onehot = ['Akt', 'SL', 'Typ', 'LA', 'WB', 'KT', 'RForm', 'SA']
df_encoded = pd.get_dummies(df, columns=columns_to_onehot)

In [40]:
df_encoded.shape #to check if one hot encoding worked

(298, 468)

In [41]:
nan_columns = df_encoded.columns[df_encoded.isnull().any()]
print("Columns with NaN values:")
print(nan_columns)
nan_sum = df_encoded[nan_columns].isnull().sum()
print("Sum of NaN values for each column:")
print(nan_sum)

Columns with NaN values:
Index([], dtype='object')
Sum of NaN values for each column:
Series([], dtype: float64)


In [44]:
#df_encoded.to_csv('/content/drive/MyDrive/Module 3/Final_Final_Lenja.csv', index=False)
df_encoded.to_csv('C:/Users/Gaëlle/Documents/_CAS applied data science/3. Module 3 Data analysis and machine learning/Project/Models/final_GMA.csv', index=False)